In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-02-05 21:47:37--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.19MB/s    in 0.5s    

2026-02-05 21:47:38 (2.19 MB/s) - ‘input.txt’ saved [1115394/1115394]



# Data

In [2]:
# read 
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print('length of dataset in characters is:', len(text))

length of dataset in characters is: 1115394


In [4]:
# look at first 1000 chars 
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# all unique chars that occur in this text 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create mapping from strings to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, output list of characters 
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of integer, outputs characters

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# encode entire text dataset and store in torch.Tensor 
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# train and val split 
n = int(0.9*len(data)) # first 90% train and then rest eval
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8 # also know as context length 
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target is {target}')

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [13]:
torch.manual_seed(1337)
batch_size = 4 # how many independent seq will be processed in parrallel? 
block_size = 8 # what is the max context length for prediction? 

# Given the previous tokens, predict the next one.
# x = [t12, t13, t14, t15, t16]
# y = [t13, t14, t15, t16, t17]
def get_batch(split):
    # generate small batch of data of input x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,)) # torch.randint(upperbound,(shape of output tensor))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size): # batch dimension 
    for t in range(block_size): # time dimension 
        context = xb[b, :t+1]
        # y is alr shifted right so t works
        target = yb[b,t]
        print(f'When input is {context} the target is {target}')


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
When input is tensor([24]) the target is 43
When input is tensor([24, 43]) the target is 58
When input is tensor([24, 43, 58]) the target is 5
When input is tensor([24, 43, 58,  5]) the target is 57
When input is tensor([24, 43, 58,  5, 57]) the target is 1
When input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
When input is tensor([44]) the target is 53
When input is tensor([44, 53]) the target is 56
When input is tensor([44, 53, 56]) the target is

In [ ]:
print(xb) # input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


# Bigram model

In [17]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        # idx and target are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) 
        # plucking out row for the specific char

        # cross entropy expects C/channel to be the 2nd dimension 
        B, T, C = logits.shape
        logits = logits.view(B*T, C) # making B*T a single dimension
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx)  # (B, T, C)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, _ = self(idx)

            # focus only on the last time step
            logits = logits[:, -1, :]  # (B, C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [22]:
# optimizer 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [27]:
batch_size = 32 
for steps in range(10000):
    # get a sample of batch 
    xb, yb = get_batch('train')

    # eval the loss 
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.502716302871704


In [29]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


To he, my YOMok tofor r, Pllanloroublled myouspibolllld'd linse cetorderu m ur it oririmes Whomo chende.
TUCUSCar destr ared o he d sot te pesfawbengorerm
RWallaclape:
Cour's hallo, t Winimaco my lc-trncint.
Ye cand br'ds,
Y:
MAulon: LENThe s s bou tave wh ithanano ndmyose. pon notherknofo wie tss cestoule llee,
Eit Bathanofas itemoman, CKEnn buen d swnitacinarn aur s er,
T; o r bnele beveoutsey S


# mathmatical trick in attention

In [30]:
# consider the following toy example 

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

## version 1 

In [31]:
# we want x[b,t] = mean_{i<t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev,dim=0) # averaging over time

In [32]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [33]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
torch.tril(torch.ones(3,3))
# lower traingle matrix

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

a is a causal averaging matrix

Row 0 → average of row 0

Row 1 → average of rows 0–1

Row 2 → average of rows 0–2

b contains data vectors

c contains prefix averages of b

In [ ]:
# trick using matrix
torch.manual_seed(42)

# Create a lower-triangular matrix of ones
# This acts like a "causal mask": each row can only see itself and previous rows
a = torch.tril(torch.ones(3, 3))

# Normalize each row so that the row sums to 1
# This turns each row into a probability distribution
# i.e. each output is an AVERAGE of allowed previous rows
a = a / torch.sum(a, dim=1, keepdim=True)

# Create a random matrix (3 rows, 2 features per row)
# Think of each row as a "value vector"
b = torch.randint(0, 10, (3, 2)).float()

# Matrix multiplication:
# Each row of c is a weighted sum (average) of rows in b
# Weights are given by the corresponding row in a
# c[i] = Σ_j a[i,j] * b[j]
c = a @ b

print("a=")
print(a)
print("---")
print("b=")
print(b)
print("---")
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


## version 2 

In [ ]:

wei = torch.tril(torch.ones(T, T)) 
wei = wei / wei.sum(dim=1, keepdim=True)
print(wei)
xbow2 = wei @ x 
# (T,T) @ (B,T,C) ---> broadcast
# (B,T,T) @ (B,T,C) ---> (B,T,C)
torch.allclose(xbow, xbow2)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [43]:
xbow2[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

# version 3 

In [47]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [48]:
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [49]:
wei = wei.masked_fill(tril == 0, float("-inf"))
wei 

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [50]:
wei = F.softmax(wei, dim=-1)
wei 

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
# use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x 
torch.allclose(xbow,xbow3)


True

# Self attention 

## version 4 

In [ ]:
# instead of equal weights distubution
# self attention assign dynamic weights and solves this issue
# so more interesting embeddings get more weightage
# attention does this by query and key embeddings
# query -> what am i looking for ? 
# key -> what do i contain ?
# value -> what gets aggregate?

In [54]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head performs self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B,T,16)
q = query(x)  # (B,T,16)
# transposing last 2 dimensions
# (B,T,16) @ (B,16,T) --> (B,T,T)
wei = q @ k.transpose(-2, -1)  # (B,T,T)

# use softmax
tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v

In [55]:
wei # diff weights 

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [ ]:
# Attention is a communication mechanism.
# Tokens can be viewed as nodes in a directed graph.

# Each token gathers information from other tokens
# via a weighted sum, where the weights are data-dependent
# (computed from queries and keys).

# Attention has NO built-in notion of space or order.
# It operates on a set of vectors.
# This is why positional encodings are required
# to inject sequence order information.

# Each example in the batch dimension is processed
# completely independently.
# Different batch elements NEVER communicate with each other.

# "Self-attention" means:
# queries, keys, and values all come from the SAME source (x).

# "Cross-attention" means:
# queries come from x,
# but keys and values come from a DIFFERENT external source
# (e.g., encoder outputs in an encoder-decoder model).

# In an ENCODER attention block:
# remove the causal (triangular) mask.
# This allows every token to communicate with every other token.

# In a DECODER attention block:
# use triangular (causal) masking.
# This prevents tokens from seeing future tokens
# and is required for autoregressive generation.

# "Scaled" dot-product attention divides attention weights
# by sqrt(head_size).

# This scaling prevents dot products from growing too large
# when Q and K have unit variance.

# Without scaling:
# softmax would saturate (become too peaky),
# hurting gradient flow.

# With scaling:
# softmax remains diffuse and stable during training.

## "Scaled" dot-product attention

In [ ]:
# Q and K have unit variance.
k = torch.randn(B,T,head_size)
q = torch.randn(B, T, head_size)
# but wei variance is on order of head_size
wei = q @ k.transpose(-2,-1) 

In [57]:
k.var()

tensor(1.0449)

In [58]:
q.var()

tensor(1.0700)

In [59]:
wei.var()

tensor(17.4690)

### if wei takes high values, it inputs softmax that converges them to similar to one-hot vectors

In [61]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 16, dim=-1)
# softmax gets peaky at highest value
# IMPORTANT TO CONTROL VARIANCE AT INITIALIZATION 

tensor([1.5939e-03, 1.3117e-05, 3.9102e-02, 1.3117e-05, 9.5928e-01])

In [ ]:
# Q and K have unit variance.
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
# but wei variance is on order of head_size
# but when multiplied by  *head_size**-0.5
wei = q @ k.transpose(-2, -1) * head_size**-0.5
# variance of wei is preserved
# it is imp to have wei variance 1 becuase 
# if wei takes high values, it inputs softmax that converges them to similar to one-hot vectors

In [67]:
k.var()

tensor(1.0966)

In [68]:
q.var()

tensor(0.9416)

In [69]:
wei.var()

tensor(1.0065)

# attention block 

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """ 
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        # wei[b, i, j] = (q[b,i] · k[b,j]) / sqrt(C)
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B,T,C) @ (B,C,T) --> (B,T,T)
        # self.tril[:T,:T] -> Slices the mask to match current input size
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1)  # (B,T,T)
        # perform weighted aggregation of values
        v = self.value(x)
        out = wei @ v  # (B,T,T) @ (B,T,C) --> (B,T,C)
        return out

# MultiHead attention